In [1]:
import requests
import pandas as pd
import geopandas as gpd
import math
from datetime import date
import datetime
import time


### Querying data from https://docs.openaq.org/docs/getting-started

In [2]:
def get_sensor_locations_in_canada( country_code = "CA", parameter = "pm25", limit = 1000):

    if(limit != 1000):
        print("Warning, it's possible that this function only works with max limit 1000, and behaves weirdly otherwise")
        
    url = f"https://api.openaq.org/v2/locations?country={country_code}&parameter={parameter}&limit=1000"
    

    headers = {"accept": "application/json", "X-API-Key": "659706290b9f942d7b7e22e0e1667afc43164846ee91949f19f5a1312d2543e4"}

    response = requests.get(url, headers=headers)
    if(response.status_code != 200):
        print(response.text)

    resj = response.json()
    
    foo = pd.DataFrame(resj['results'])
    tot_pages = resj['meta']['found']
    print(tot_pages)
    div = tot_pages/limit
    pages = math.ceil(div)
    
    df_raw = pd.DataFrame()
    
    for i in range(1, (pages+2)):
        print(i)
        url = f"https://api.openaq.org/v2/locations?country={country_code}&parameter={parameter}&limit={limit}&page={i}"
        response2 = requests.get(url, headers=headers)
        test = response2.json()
        tmp_frame = pd.DataFrame(test['results'])
        df_raw = pd.concat([ df_raw,  tmp_frame], ignore_index = True, sort = False)
        
    return(df_raw)

def unpack(df, column, fillna=None):
    ret = None
    if fillna is None:
        ret = pd.concat([df, pd.DataFrame((d for idx, d in df[column].items()))], axis=1)
        del ret[column]
    else:
        ret = pd.concat([df, pd.DataFrame((d for idx, d in df[column].items())).fillna(fillna)], axis=1)
        del ret[column]
    return ret
    

In [14]:
class CustomException(Exception):
    pass


def get_measument_per_location(locations_id, limit = 1000):
    time.sleep(3)
    if(limit != 1000):
        print("Warning, it's possible that this function only works with max limit 1000, and behaves weirdly otherwise")
        
    url = f"https://api.openaq.org/v2/averages?parameters_id=2&date_to=2023-09-30T00%3A00%3A00&date_from=2023-05-01T00%3A00%3A00&country=CA&locations_id={locations_id}&limit=1&page=1"

    headers = {"accept": "application/json", "X-API-Key": "659706290b9f942d7b7e22e0e1667afc43164846ee91949f19f5a1312d2543e4"}

    response = requests.get(url, headers=headers)
    
    if(response.status_code != 200):
        print(response.text)
        if(response.status_code == 429):
            return(None)
        

    test = response.json()
    
    df_raw = pd.DataFrame()
    if(test['meta']['found'] == 0):
        print(test)
        print("no entries found for id", locations_id)
        raise CustomException(" ")
    page = 1
    while ((test['meta']['found'] != 0)):
        time.sleep(1)
        url = f"https://api.openaq.org/v2/averages?parameters_id=2&date_to=2023-09-30T00%3A00%3A00&date_from=2023-05-01T00%3A00%3A00&country=CA&locations_id={locations_id}&limit=1000&page={page}"
        response2 = requests.get(url, headers=headers)
        if(response2.status_code != 200):
            print(response2.text)
            if(response2.status_code == 429):
                return(None)
        test = response2.json()
        
        tmp_frame = pd.DataFrame(test['results'])
        df_raw = pd.concat([ df_raw,  tmp_frame], ignore_index = True, sort = False)
        page = page + 1
        
        
    return(df_raw)

def get_measument_per_location_special(locations_id, limit = 1000):
    '''
    Get measurment per location. Different than previous function because it will query over a larger timeframe, and then subset the result. 
    '''
    if(limit != 1000):
        print("Warning, it's possible that this function only works with max limit 1000, and behaves weirdly otherwise")
        
    url = f"https://api.openaq.org/v2/averages?parameters_id=2&date_to=2024-02-08T00%3A00%3A00&date_from=2023-05-01T00%3A00%3A00&country=CA&locations_id={locations_id}&limit=1&page=1"

    headers = {"accept": "application/json", "X-API-Key": "659706290b9f942d7b7e22e0e1667afc43164846ee91949f19f5a1312d2543e4"}

    response = requests.get(url, headers=headers)
    
    if(response.status_code != 200):
        print(response.text)
        if(response.status_code == 429):
            return(None)
        

    test = response.json()
    
    df_raw = pd.DataFrame()
    if(test['meta']['found'] == 0):
        print(test)
        print("no entries found for id", locations_id)
        raise CustomException(" ")
    page = 1
    while ((test['meta']['found'] != 0)):
        url = f"https://api.openaq.org/v2/averages?parameters_id=2&date_to=2024-02-08T00%3A00%3A00&date_from=2023-05-01T00%3A00%3A00&country=CA&locations_id={locations_id}&limit=1000&page={page}"
        response2 = requests.get(url, headers=headers)
        if(response2.status_code != 200):
            print(response2.text)
            if(response2.status_code == 429):
                return(None)
        test = response2.json()
        
        tmp_frame = pd.DataFrame(test['results'])

        df_raw = pd.concat([ df_raw,  tmp_frame], ignore_index = True, sort = False)
        page = page + 1
        
    df_raw[(df_raw.hour <= "2023-09-30 23:59:59") & (df_raw.hour >= "2023-05-01 00:00:00")]
    return(df_raw)

In [7]:
locs = get_sensor_locations_in_canada()

locs.to_csv("/projects/old_shared/Smoke_over_Canada_2023/openAQ_measurements/" + "Locations" + str(date.today().strftime("%Y%m%d")) +".csv")
print(len(locs.id.unique()))

2276
1
2
3
4
2268


In [8]:
### Find number of IDS in the time range. (Not actually sure how strict the querying is, it might not drop IDS based on if they only partially overlap. Need to check). 
start_time = "2023-05-01 00:00:00+00:00"
end_time = "2023-09-30 00:00:00+00:00"




len(locs[(locs.firstUpdated.astype('datetime64[ns, UTC]') <= start_time) & (locs.lastUpdated.astype('datetime64[ns, UTC]') >= end_time)].id.unique())

locs_time = locs[(locs.firstUpdated.astype('datetime64[ns, UTC]') <= start_time) & (locs.lastUpdated.astype('datetime64[ns, UTC]') >= end_time)]

In [15]:
ids = locs.id.unique()
ids = ids[::-1]

all_measurments = pd.DataFrame()
foo = pd.DataFrame()
no_entries_id = list()


for n,i in enumerate(ids, start = 0):
    if(n == 0):
        print("n = 0")
    elif((n%150) == 0):
        time.sleep(3)
    try:
        foo = get_measument_per_location_special(i)
    except CustomException as e:
        no_entries_id.append(i)
        
    except Exception as e:
        print("Error at ID: ",i)
        print(e)
        continue
    ## Extract the period between 
    all_measurments = pd.concat([all_measurments, foo])
    #print(fires)
        #fr_pd = pd.DataFrame(fires, columns=["lat", "lon", "farea", "data_source"])
    all_measurments.to_csv("/projects/old_shared/Smoke_over_Canada_2023/openAQ_measurements/" + str(date.today().strftime("%Y%m%d"))+  ".csv")
    tmp_df = pd.DataFrame(no_entries_id, columns= ["none_found"])
    tmp_df.to_csv("/projects/old_shared/Smoke_over_Canada_2023/openAQ_measurements/" + "missing_ids"+ str(date.today().strftime("%Y%m%d"))+  ".csv")

n = 0
{'meta': {'name': 'openaq-api', 'license': '', 'website': '/', 'page': 1, 'limit': 1, 'found': 0}, 'results': []}
no entries found for id 529
{'meta': {'name': 'openaq-api', 'license': '', 'website': '/', 'page': 1, 'limit': 1, 'found': 0}, 'results': []}
no entries found for id 754
{'meta': {'name': 'openaq-api', 'license': '', 'website': '/', 'page': 1, 'limit': 1, 'found': 0}, 'results': []}
no entries found for id 1065
{'meta': {'name': 'openaq-api', 'license': '', 'website': '/', 'page': 1, 'limit': 1, 'found': 0}, 'results': []}
no entries found for id 1141358
{'meta': {'name': 'openaq-api', 'license': '', 'website': '/', 'page': 1, 'limit': 1, 'found': 0}, 'results': []}
no entries found for id 6138
{'meta': {'name': 'openaq-api', 'license': '', 'website': '/', 'page': 1, 'limit': 1, 'found': 0}, 'results': []}
no entries found for id 527
{'meta': {'name': 'openaq-api', 'license': '', 'website': '/', 'page': 1, 'limit': 1, 'found': 0}, 'results': []}
no entries found for i

In [9]:
### Read in missing IDS from previous runs. These need to be handled differently becuase of the bug in the API

missing_ids = pd.read_csv("/projects/old_shared/Smoke_over_Canada_2023/openAQ_measurements/missing_ids20240207.csv")

missing_ids

,Unnamed: 0,none_found
0,0,529
1,1,754
2,2,1065
3,3,1141358
4,4,6138
...,...,...
538,538,325972
539,539,1656251
540,540,1192970
541,541,1552550


In [5]:
## Check file to see if it processed all the IDS. If not, subset to missing IDs and try again. 

# sensors = pd.read_csv("/projects/old_shared/Smoke_over_Canada_2023/openAQ_measurements/20240125.csv")
# sensors2 = pd.read_csv("/projects/old_shared/Smoke_over_Canada_2023/openAQ_measurements/20240126.csv")
# sensor3 = pd.read_csv("/projects/old_shared/Smoke_over_Canada_2023/openAQ_measurements/20240129.csv")
# sensor4 = pd.read_csv("/projects/old_shared/Smoke_over_Canada_2023/openAQ_measurements/20240130.csv")
# sensor5 = pd.read_csv("/projects/old_shared/Smoke_over_Canada_2023/openAQ_measurements/20240201.csv")
sensor6 = pd.read_csv("/projects/old_shared/Smoke_over_Canada_2023/openAQ_measurements/20240207.csv")



In [10]:
### Check just on the IDS where there is temporal overlap 

#sub_missing_ids = locs_time.id[~locs_time.id.isin([*sensors.id.unique(), *sensors2.id.unique(), *sensor3.id.unique(),  *sensor5.id.unique(), *sensor6.id.unique()])]
sub_missing_ids = locs_time.id[~locs_time.id.isin([ *sensor6.id.unique()])]

In [11]:
print(sub_missing_ids[~sub_missing_ids.isin(missing_ids.none_found)])
print(~missing_ids.none_found[missing_ids.none_found.astype("str").isin(sub_missing_ids)])

new_missing_ids = [*missing_ids.none_found, *sub_missing_ids[~sub_missing_ids.isin(missing_ids.none_found)]]
#new_missing_ids

3        222466
34        66605
149      355710
158      371237
189      345860
202       68708
212      356053
248      276957
269      274513
315      235881
318      275853
385      236365
395      561758
403      273457
415       65084
423      232235
437      276320
450      635657
462      294383
478      219293
480      224459
492      236351
544      351732
582     1966742
599      276817
622      354565
667      337006
687      222032
701      367264
773       61990
802      221561
918       62365
933      222033
938      500721
946       62105
954      345237
962      236191
963      322667
973      350574
1009     236030
1035        504
1095       8654
1096       8683
1129     218369
1133     269357
1138     342749
1165       1138
1187       1179
1202        546
1279      69875
1319     227443
1371     231477
Name: id, dtype: int64
Series([], Name: none_found, dtype: int64)


In [15]:
all_measurments = pd.DataFrame()
no_entries_id = list()
foo = pd.DataFrame()

for n,i in enumerate(new_missing_ids, start = 0):
    if(n == 0):
        print("n = 0")
    elif((n%150) == 0):
        time.sleep(3)
    try:
        foo = get_measument_per_location_special(i)
    except CustomException as e:
        no_entries_id.append(i)
        
    except Exception as e:
        print("Error at ID: ",i)
        print(e)
        continue
    ## Extract the period between 
    all_measurments = pd.concat([all_measurments, foo])
    #print(fires)
        #fr_pd = pd.DataFrame(fires, columns=["lat", "lon", "farea", "data_source"])
    all_measurments.to_csv("/projects/old_shared/Smoke_over_Canada_2023/openAQ_measurements/" + str(date.today().strftime("%Y%m%d"))+  ".csv")
    tmp_df = pd.DataFrame(no_entries_id, columns= ["none_found"])
    tmp_df.to_csv("/projects/old_shared/Smoke_over_Canada_2023/openAQ_measurements/" + "missing_ids"+ str(date.today().strftime("%Y%m%d"))+  ".csv")
    
    

n = 0
{'meta': {'name': 'openaq-api', 'license': '', 'website': '/', 'page': 1, 'limit': 1, 'found': 0}, 'results': []}
no entries found for id 529
{'meta': {'name': 'openaq-api', 'license': '', 'website': '/', 'page': 1, 'limit': 1, 'found': 0}, 'results': []}
no entries found for id 754
{'meta': {'name': 'openaq-api', 'license': '', 'website': '/', 'page': 1, 'limit': 1, 'found': 0}, 'results': []}
no entries found for id 1065
{'meta': {'name': 'openaq-api', 'license': '', 'website': '/', 'page': 1, 'limit': 1, 'found': 0}, 'results': []}
no entries found for id 1141358
{'meta': {'name': 'openaq-api', 'license': '', 'website': '/', 'page': 1, 'limit': 1, 'found': 0}, 'results': []}
no entries found for id 6138
{'meta': {'name': 'openaq-api', 'license': '', 'website': '/', 'page': 1, 'limit': 1, 'found': 0}, 'results': []}
no entries found for id 527
{'meta': {'name': 'openaq-api', 'license': '', 'website': '/', 'page': 1, 'limit': 1, 'found': 0}, 'results': []}
no entries found for i

In [162]:
# url = "https://api.openaq.org/v2/measurements?date_from=2023-05-01T00%3A00%3A00&date_to=2023-09-30T00%3A00%3A00&limit=7000&page=50&offset=0&sort=desc&has_geo=true&parameter_id=2&radius=1000&country=CA&order_by=datetime&entity=null"

# headers = {"accept": "application/json"}

# response = requests.get(url, headers=headers)

# test = response.json()
# measurements = pd.DataFrame(test['results'])

In [128]:
### Put data into useful forms for write-out

all_sens = pd.concat([sensors, sensors2, sensor3, sensor4, sensor5, sensor6], ignore_index = True)
all_sens = all_sens.drop_duplicates()


## Unpack nested column information
locs = unpack(locs, 'coordinates')
tmp = unpack(locs, 'manufacturers')
tmp = tmp.rename(columns={0:"manufacturers"})
locs = unpack(tmp, 'manufacturers')

## Write out location data

locs.to_csv("/projects/old_shared/Smoke_over_Canada_2023/openAQ_measurements/" + "Locations_formatted.csv") 


locs = locs[["id", "city","lastUpdated", "firstUpdated", "measurements", "latitude", "longitude", "modelName", "manufacturerName"]] # Subset to columns that are more informative for merge

all_sens = all_sens[['id', 'name', 'hour', 'day', 'month', 'year', 'hod',
       'dow', 'average', 'measurement_count', 'parameter', 'parameterId',
       'displayName', 'unit', 'first_datetime', 'last_datetime']] # Also subset to infotmative colmns
all_sens = all_sens.drop_duplicates()

### Put data into formates to prepare for merging
all_sens = all_sens[['id', 'name', 'hour','average', 'measurement_count', 'parameter',
       'displayName', 'unit']]
all_sens.city = all_sens.city.astype('str')
all_sens.id = all_sens.id.astype("float").apply(math.floor)
all_sens.id = all_sens.id.astype("str")
locs.id = locs.id.astype("str")

all_sens = all_sens.merge(locs, on = "id", how = "left")

In [146]:
### Group by ID to get summary statistics

sens_mean = all_sens.groupby(["id",'name','parameter',
       'displayName', 'unit', 'city', 'lastUpdated', 'firstUpdated',
       'latitude', 'longitude', 'modelName',
       'manufacturerName' ]).average.mean()
sens_mean = sens_mean.reset_index()

max_sens = all_sens.groupby(["id"]).average.max()
max_sens = max_sens.reset_index()
max_sens = max_sens.rename(columns={"average":"max"})

sens_mean = sens_mean.merge(max_sens, on = ["id"])

mean_sens = all_sens.groupby(["id"]).average.mean()
mean_sens = mean_sens.reset_index()
mean_sens = mean_sens.rename(columns={"average":"mean"})

sens_mean = sens_mean.merge(mean_sens, on = ["id"])

quant_upper = all_sens.groupby(["id"]).average.quantile(0.975)
quant_upper = quant_upper.reset_index()
quant_upper = quant_upper.rename(columns={"average":"quant_97.5"})

sens_mean = sens_mean.merge(quant_upper, on = ["id"])

quant_lower = all_sens.groupby(["id"]).average.quantile(0.025)
quant_lower = quant_lower.reset_index()
quant_lower = quant_lower.rename(columns={"average":"quant_02.5"})

sens_mean = sens_mean.merge(quant_lower, on = ["id"])

quant_25 = all_sens.groupby(["id"]).average.quantile(0.25)
quant_25 = quant_25.reset_index()
quant_25 = quant_25.rename(columns={"average":"quant_25"})

sens_mean = sens_mean.merge(quant_25, on = ["id"])

quant_75 = all_sens.groupby(["id"]).average.quantile(0.75)
quant_75 = quant_75.reset_index()
quant_75 = quant_75.rename(columns={"average":"quant_75"})

sens_mean = sens_mean.merge(quant_75, on = ["id"])

min_sens = all_sens.groupby(["id"]).average.min()
min_sens = min_sens.reset_index()
min_sens = min_sens.rename(columns={"average":"min"})

sens_mean = sens_mean.merge(min_sens, on = ["id"])

In [159]:
sens_mean = sens_mean.drop("average", axis = 1)

In [163]:
sens_mean.to_csv("/projects/old_shared/Smoke_over_Canada_2023/openAQ_measurements/" + "Summary_statistics_pm2.5_sensors_in_CA.csv")